In [ ]:
#LOAD NEEDED PACKAGES
import statsmodels.api as sm
import pandas as pd
import numpy as np
import gme as gme
import matplotlib.pyplot as plt
import geopandas as gpd
import folium
from mpl_toolkits.axes_grid1 import make_axes_locatable
import gegravity as ge

#SHOW ALL COLUMNS
pd.set_option('display.max_colwidth', None)
#CHANGE FORMAT OF FLOATS
pd.set_option('display.float_format', '{:20,.10f}'.format)


#LOAD SHP MAPS
mapa_woj = gpd.read_file(r'SHP\Województwa.shp')
mapa_woj = mapa_woj[['JPT_NAZWA_','geometry']]

#LOAD DATA
df_init = pd.read_csv('https://media.githubusercontent.com/media/mikolajnowak97/MasterThesis_The-use-of-the-gravity-model-in-logistics/main/Data/04_trips_calculated.csv', sep=';')

#TRANSFORM COLUMNS
df_init['year'] = 2021
df_init['distance_km'] = np.where(df_init['distance_m']>0, df_init['distance_m']/1000, 0) 
df_init['toll_distance_km'] = np.where(df_init['toll_distance_m']>0, df_init['toll_distance_m']/1000, 0)
df_init['toll_cost_euro'] = np.where(df_init['toll_cost_euro']>0, df_init['toll_cost_euro']/100, 0)
df_init['time_h'] = np.where(df_init['time_s']>0, df_init['time_s']/3600, 0)

#CHOOSE COLUMNS TO MODEL
df = df_init[['from_province', 'to_province', 'distance_m', 'distance_km', 'toll_distance_m', 'toll_distance_km',
              'toll_cost_euro', 'time_s', 'time_h', 'manoeuvres', 'hydrocarbons_g', 
              'methane_g', 'hydrocarbonsExMethane_g', 'carbonMonoxide_g', 'carbonDioxide_g', 'sulphurDioxide_g', 
              'nitrogenOxides_g', 'nitrousOxide_g', 'ammonia_g', 'benzene_g', 'particles_g', 'fuel_kg', 'year']]

#DEFINE MODEL VARIABLES
variables = ['hydrocarbons_g', 'methane_g', 'carbonMonoxide_g', 'carbonDioxide_g', 'sulphurDioxide_g',
       'nitrogenOxides_g', 'nitrousOxide_g', 'ammonia_g', 'benzene_g',
       'particles_g', 'fuel_kg']


#CREATE GRAVITY MODEL FOR EACH VARIABLE
for variable in variables:
    gme_data = gme.EstimationData(data_frame = df,
                                  imp_var_name = 'from_province',
                                  exp_var_name = 'to_province',
                                  trade_var_name = variable,
                                  year_var_name = 'year')

    model = gme.EstimationModel(estimation_data = gme_data,
                                lhs_var = variable,
                                rhs_var = ['distance_km',
                                           'time_h',
                                           'toll_distance_km',
                                           'toll_cost_euro',
                                           'manoeuvres'],
                                fixed_effects = ['from_province','to_province'])
    estimates = model.estimate()
    results = estimates['all']

    df_coef = results.params.to_frame().rename(columns={0: 'coef'})
    df_se = results.bse.to_frame().rename(columns={0: 'se'})
    df_p = results.pvalues.to_frame().rename(columns={0: 'p_values'})

    df_summary = df_coef.merge(df_se, left_index=True, right_index=True).merge(df_p, left_index=True, right_index=True)

    print(results.summary())
    with open('outputs/'+variable+'_summary.txt', 'w', encoding='utf-8') as f:
        f.write(results.summary().as_latex())
    print(model.format_regression_table())
    with open('outputs/'+variable+'_format_regression_table.txt', 'w', encoding='utf-8') as f:
        f.write(model.format_regression_table().to_latex())

    df_summary.reset_index(level=0, inplace=True)
    df_summary = df_summary.rename(columns={'index': 'woj'})
    data = df_summary.copy()
    data = data[data['woj'].str.contains('from_province_fe_', regex= True, na=False)].replace(to_replace ='from_province_fe_', value = '', regex = True)
    data_map = pd.merge(mapa_woj, data, how='left', left_on='JPT_NAZWA_', right_on='woj')

    data_map = data_map.to_crs(epsg=2180)

    fix, ax = plt.subplots(1, figsize=(8,8))
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.1)
    data_map.plot(column='coef',
                     ax=ax,
                     cax=cax,
                     cmap='YlOrRd',
                     linewidth=0.8,
                     edgecolor='gray',
                     legend=True)
    ax.axis('off')
    plt.savefig('outputs/'+variable+'_coef.png')